In [3]:
!pip install scikit-learn
# 아이리스
from sklearn.datasets import load_iris
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
# 평가지표
from sklearn.metrics import accuracy_score as acc_sc  # alias


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 만약 데이터를 쪼개지 않고 통으로 한다면,

In [4]:
iris = load_iris()

X_data = iris.data # X = 독립변수 = 설명변수 = feature(머신러닝) 
y_target = iris.target # y = 종속변수(이산) = target = label

## 의사결정나무 객체화 
dt_clf = DecisionTreeClassifier()  # Black Box 설정 == 알고리즘 (어떤 규칙을 찾음)

In [5]:
X_train = X_data.copy()
y_train = y_target.copy()

# 학습 수행 - 데이터 분할 X
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [6]:
# 예측 수행
y_pred = dt_clf.predict(X_train)  # 2차원의 입력 X값이 들어와야 한다.

In [7]:
# 평가지표로 평가 - acc_sc
print(f'예측 정확도: {acc_sc(y_train, y_pred)}')

예측 정확도: 1.0


### '예측 정확도: 1.0' 이 나온 이유
원래는 X_data - (Black Box) - Y_target 구성이었음.

Black Box 안에다가 X_train & y_train 를 넣었음.

여기서 X_train은 모든 독립변수 각각을 모두 합한 상태를 복사해서 넣었기 때문에 y_target과 y_pred 값이 당연히 같게 나옴 (acc_sc = 1)

따라서 학습을 시킬 때, X_data에서 X_test를 추출하고 y_train에서 y_test를 추출해서 분리 데이터를 만들고 -> Black Box에 넣어서 predict -> 'y_test = y_actual'가 되고, X_test로 예측한 y_pred가 나옴 -> y_pred와 y_test(ay_actual)를 비교! 

### 데이터를 쪼개서 예측할 때,

In [8]:
from sklearn.model_selection import train_test_split
# 이렇게 train과 test 두 가지를 분리하는 것을 Hold_out(홀드아웃)이라고 함.

In [16]:
import numpy as np
array_1 = np.arange(1,11)
array_11 = np.arange(11,21)

### 연습
# train_test_split(array_1, test_size=0.2)  # default=25%, train값은 50% 이상이어야 함.
# 이 상태에서는 클릭할 때마다 값이 바뀜. 만약 고정시키고 싶다면,

# train1, test1 = train_test_split(array_1, test_size=0.2, random_state=121)  # 보통은 random_state 값은 400 미만의 숫자 아무거나 
# train11, test11 = train_test_split(array_11, test_size=0.2, random_state=121) 
# # 위 두줄을 한 줄로 줄이면,
train1, test1, train11, test11 = train_test_split(array_1, array_11, test_size=0.2, random_state=121)  # 보통은 random_state 값은 400 미만의 숫자 아무거나 

In [17]:
# 다시 본문으로 돌아와서..
# X와 y 분할
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=121)

In [18]:
## 재학습
dt_clf.fit(X_train, y_train)

## 예측 수행
y_pred = dt_clf.predict(X_test)

In [19]:
## 평가지표
acc_sc(y_test, y_pred)

0.9666666666666667

In [20]:
print(f'예측정확도: {np.round(acc_sc(y_test, y_pred), 3)}')

예측정확도: 0.967


## Validation (교차검증)

#### 과적합(Overfitting)

현재 우리는 'test_size=0.2, random_state=121'로 고정시켜서 했다.

이 경우, 과적합(overfitting)의 위험에서 벗어날 수 없다. (과적합되는 것을 피해야 함.)

따라서 '교차검증'을 통해서 해결해보자.

그렇다면, '교차검증'이란 무엇인가?
    
    - Validation을 여러번 하는 것. --> validation을 한 번만 하면 overfitting될 수 있어서 여러번 하는 것!
    - 그렇다면, Validation을 어떻게 할 것인가? ex) k-fold 방법 :: k를 내가 정할 수 있다. 
    - * (마치 종이접기하는 것처럼 전체 train 데이터를 k개로 나누기)


#### 참고)

- test 데이터는 만들고 나서는 절대 건들지 말 것
- train 데이터를 쪼개서 다른 test 데이터를 만들기 ==> Validation 데이터 생성 ==> 그러면 original train = train(2/3만큼) 데이터 + validation 데이터
- Validation 데이터를 가지고 Black Box에 넣어서 학습 시킨다. 이 때, train(2/3) 데이터는 학습한 후에 삭제.
- Validation 데이터로 학습시킨 결과값인 pred_val를 test 데이터와 비교

#### 교차검증 과정 설명

- k := 하이퍼파라미터 (사람이 정하는 것)
- k = 5 로 지정하면, original train에서 'train 데이터 + validation 데이터' 세트가 5개로 쪼개짐. (보통은 10번 돌림.)

- CV := Cross Validation (교차검증) ==> 사람이 데이터를 섞어서 검증하는 것

    - CV = 1 := 첫번째 k 데이터셋을 validation으로, 나머지 4개를 train으로.
    - CV = 2 := 두번째 k 데이터셋을 validation으로, 나머지 4개를 train으로.
    - ... 
    - CV = 5 까지 진행

총 5번의 validation이 수행됨.

따라서, k = CV = 5

즉, k는 내가 정하고, CV는 k와 동일하게 설정되는 것.


##### test 데이터를 건드릴 수 없으니, train 데이터에서 validation 데이터를 통해 가상의 test를 만들어 진행하고, black box의 알고리즘을 통해 predict하는 것.

##### test와 train 데이터 안에는 x, y 모두 있음.

In [38]:
## Validation 설정

# 라이브러리 호출 순서 (권장)
# 데이터 관련 함수 - 모델 셀렉션(model_selection 관련: train_test_split) - 필요한 알고리즘 호출: 의사결정나무, 랜덤포레스트 

# 필수 라이브러리 호출
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold  # overfitting 피하려고 사용하는 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc_sc

import numpy as np
import pandas as pd

In [40]:
# 데이터 재정의: 앞서 했던 거 말고 새롭게!

iris = load_iris()
features = iris.data # X값
label = iris.target  # y값

# 알고리즘 객체화
dt_clf = DecisionTreeClassifier(random_state=156)

# kfold에서 k = 5
kfold = KFold(n_splits=5)
print('데이터셋의 크기:', features.shape[0])

데이터셋의 크기: 150


In [51]:
cv_accuracy = []
n_iter = 0

for train_index, val_index in kfold.split(label):
    # Kfold.split()으로 반환된 인덱스를 통해 학습용, 검증용 데이터 정의
    X_train, X_val = features[train_index], features[val_index]
    y_train, y_val  = label[train_index], label[val_index]
    
    # 각 CV별 학습 및 예측
    dt_clf.fit(X_train,y_train) # 학습
    pred = dt_clf.predict(X_val) # 예측
    
    # 정확도 지표 계산
    accuracy = np.round(acc_sc(y_val, pred), 3)
    train_size = X_train.shape[0]
    val_size = X_val.shape[0]
    n_iter += 1
    print(f'\n #{n_iter} CV 정확도: {accuracy}, 학습데이터 크기: {train_size}, 검증데이터 크기: {val_size}')
    print(f'#{n_iter} 검증데이터셋 인덱스: {val_index}')
    cv_accuracy.append(accuracy)

# 개별 iteration 별 정확도 평균내기
print(f'\n ## 평균 CV 정확도: {np.mean(cv_accuracy)}')


 #1 CV 정확도: 1.0, 학습데이터 크기: 120, 검증데이터 크기: 30
#1 검증데이터셋 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

 #2 CV 정확도: 0.967, 학습데이터 크기: 120, 검증데이터 크기: 30
#2 검증데이터셋 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

 #3 CV 정확도: 0.867, 학습데이터 크기: 120, 검증데이터 크기: 30
#3 검증데이터셋 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

 #4 CV 정확도: 0.933, 학습데이터 크기: 120, 검증데이터 크기: 30
#4 검증데이터셋 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

 #5 CV 정확도: 0.733, 학습데이터 크기: 120, 검증데이터 크기: 30
#5 검증데이터셋 인덱스: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

 ## 평균 CV 정확도: 0.9
